In [1]:
import json
import sys
import os
import importlib
import re
from pydantic import BaseModel

In [3]:
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\paolo\\Desktop\\Symbolic-AI-Vs-LLMs\\utils.py'>

In [2]:
sys.path.append(os.path.abspath("C:/Users/paolo/Desktop/Symbolic-AI-Vs-LLMs"))

import utils

In [4]:
KNAPSACK_PROMPT_BASE_4O = """
You have a set of items. Each item has a weight and a value. You have a knapsack that has a maximum capacity. Your goal is to maximize the value of the items you carry in the knapsack without exceeding the maximum capacity (the subset you select should be that with the maximum total value which is less than or equal to the maximum capacity). In the answer, include a string with the reasoning you used to find the solution and a json string with the solution in the form specified below.

Items: ###
{}
###

MaximumCapacity: ###
{}
###

Desired format: ###
Reasoning: ....

Solution: 
[
    {{
        "Name": "....",
        "Weight": ...,
        "Value": ...
    }},
    {{
        "Name": "....",
        "Weight": ...,
        "Value": ...
    }},
    ....
]
###
"""

In [5]:
class KnapsackItemSchema(BaseModel):
    name: str
    weight: int
    value: int

class KnapsackOutputSchema(BaseModel):
    reasoning: str
    solution: list[KnapsackItemSchema]



In [6]:
def knapsack_to_llm_adapter(prompt):
    """
    Input: items of the instance of knapsack problem as defined in /utils.py
    Output: prompt formatted with the items converted to a json string and the capacity
    """
    instance = utils.get_knapsack_instance()
    return prompt.format(json.dumps(instance["items"]), instance["capacity"])

In [7]:
KNAPSACK_4O_CONVERSATION = [
    {"role": "system", "content": "You have to solve the knapsack problem."},
    {"role": "user", "content": knapsack_to_llm_adapter(KNAPSACK_PROMPT_BASE_4O)}
]


In [9]:
client = utils.get_openai_client()
response = utils.interrogate_4o(client, "mini", KNAPSACK_4O_CONVERSATION, KnapsackOutputSchema)
print(response)

reasoning='To solve the knapsack problem with a maximum capacity of 20, I will evaluate different combinations of items to maximize the total value without exceeding the weight limit. I will use a methodical approach to determine the optimal selection of items based on their value-to-weight ratio, ensuring the combined weight does not exceed 20. After testing various combinations, the selection of items that maximizes the total value while remaining within the weight constraint emerges clearly, which includes valuable items with relatively lower weights.' solution=[KnapsackItemSchema(name='Diamond Necklace', weight=2, value=1200), KnapsackItemSchema(name='First Edition Charizard Card', weight=1, value=1500), KnapsackItemSchema(name='Antique Clock', weight=5, value=1200), KnapsackItemSchema(name='Lightsaber Replica', weight=3, value=1200), KnapsackItemSchema(name='Wedding Crown', weight=3, value=2000), KnapsackItemSchema(name='Vault Key', weight=1, value=1000), KnapsackItemSchema(name='

In [8]:
KNAPSACK_PROMPT_BASE_O1 = """
You have a set of items. Each item has a weight and a value. You have a knapsack that has a maximum capacity. Your goal is to maximize the value of the items you carry in the knapsack without exceeding the maximum capacity. In the answer, include a json string with the solution in the form specified below. Include only the json string in the form below, without any additional information. 

Items: ###
{}
###

MaximumCapacity: ###
{}
###

Desired format: ###
[
    {{
        "Name": "....",
        "Weight": ...,
        "Value": ...
    }},
    {{
        "Name": "....",
        "Weight": ...,
        "Value": ...
    }},
]
###
"""

In [9]:
KNAPSACK_O1_CONVERSATION = [
    {"role": "user", "content": knapsack_to_llm_adapter(KNAPSACK_PROMPT_BASE_O1)}
]

In [10]:
def knapsack_checking_schema(response):
    """
    Boolean function that checks if the response provided by a LLM without the response format feature adheres to the required schema.
    Includes a pre-processing step where everything which is not contained in the square brackets is removed.
    Such a pre-processing step has been derived from the observation that the LLM sometimes includes additional information in the response.
    
    If the check is successful, the function returns the response in the correct format.
    Otherwise, it returns False.

    Input: string response provided by the LLM
    """
    match = re.search(r'(\[.*\])', response, re.DOTALL)  #  pre-processing step
    if not match:
        return False
    response = match.group(1)
    
    try:
        response = json.loads(response)
    except:
        return False
    if not isinstance(response, list):
        return False
    
    for item in response:
        if not isinstance(item, dict):
            return False
        if "Name" not in item or not isinstance(item["Name"], str):
            return False
        if "Weight" not in item or not isinstance(item["Weight"], (int, float)):
            return False
        if "Value" not in item or not isinstance(item["Value"], (int, float)):
            return False
         
    return response

In [11]:
client = utils.get_openai_client()
response = utils.interrogate_o1(client, "mini", KNAPSACK_O1_CONVERSATION, knapsack_checking_schema)
print(response)


```json
[
    {
        "Name": "Gold Bar",
        "Weight": 10,
        "Value": 2500
    },
    {
        "Name": "Wedding Crown",
        "Weight": 3,
        "Value": 2000
    },
    {
        "Name": "First Edition Charizard Card",
        "Weight": 1,
        "Value": 1500
    },
    {
        "Name": "Diamond Necklace",
        "Weight": 2,
        "Value": 1200
    },
    {
        "Name": "Pokéball Collector's Edition",
        "Weight": 1,
        "Value": 600
    },
    {
        "Name": "Gold Chain",
        "Weight": 2,
        "Value": 600
    },
    {
        "Name": "Golden Watch",
        "Weight": 1,
        "Value": 500
    }
]
```


JSONDecodeError: Expecting value: line 1 column 1 (char 0)